In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision

import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import numpy as np
import torch.nn.functional as F
from tqdm import tqdm
from dataset import MNIST_Moving

from encoder import Embedded_Encoder
from decoder import Embedded_Decoder
from conv_lstm import ConvLSTM

In [2]:
train_set = MNIST_Moving(root='.data/mnist', train=True, download=True)
test_set = MNIST_Moving(root='.data/mnist', train=False, download=True)

batch_size = 100

train_loader = torch.utils.data.DataLoader(
                 dataset=train_set,
                 batch_size=batch_size,
                 
                 shuffle=True)
test_loader = torch.utils.data.DataLoader(
                dataset=test_set,
                batch_size=batch_size,
                shuffle=False)

In [3]:
input= next(iter(train_loader))
print(input.shape)
vid_seq = input[0]
print(vid_seq.shape)

torch.Size([100, 20, 1, 64, 64])
torch.Size([20, 1, 64, 64])


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [5]:
# ModelEncoder = Embedded_Encoder(device)
# print("The input is of the size",vid_seq.shape)

# encoderLayerOutputs=ModelEncoder(vid_seq.float().cuda())
# #These features or outputs from each encoder can be then passed on to the convlstm 

# print("The outputs of each encoder in the embedded encoder:")
# for ftr in encoderLayerOutputs: print(ftr.shape)


# ModelDecoder=Embedded_Decoder()
# print("The input of the embedded decoder is the output from the last encoder in the embedded encoder",encoderLayerOutputs[-1].shape)
# decoderLayerOutputs=ModelDecoder(encoderLayerOutputs[-1])
# print("The outputs of each encoder in the embedded decoder:")
# for ftr in decoderLayerOutputs: print(ftr.shape)

In [6]:
conv_model_FirstEncoder = ConvLSTM(input_dim= 64, hidden_dim = 64, kernel_size = (5,5), num_layers= 2)
if torch.cuda.is_available():
    conv_model_FirstEncoder.to(device)

conv_model_SecondEncoder= ConvLSTM(input_dim= 128, hidden_dim = 128, kernel_size = (5,5), num_layers= 2)
if torch.cuda.is_available():
    conv_model_SecondEncoder.to(device)

conv_model_ThirdEncoder= ConvLSTM(input_dim= 256, hidden_dim = 256, kernel_size = (5,5), num_layers= 2)
if torch.cuda.is_available():
    conv_model_ThirdEncoder.to(device)

In [7]:
abc = torch.rand((1, 64, 16, 16))
abc_3 = torch.rand((1, 256, 4, 4))

llm = conv_model_FirstEncoder(abc)
llm_3 = conv_model_ThirdEncoder(abc_3)

In [24]:
device = "cuda"
def training(criterion):
#     Seq of 20 images
    input_seq = torch.rand(((20, 1, 64, 64)))
    embd_model = Embedded_Encoder(device)
    embds = embd_model(input_seq.float().cuda())
    lstm_1, (h_1, c_1) = conv_model_FirstEncoder(embds[0])
    lstm_2, (h_2, c_2) = conv_model_SecondEncoder(embds[1])
    lstm_3, (h_3, c_3) = conv_model_ThirdEncoder(embds[2])
    
    print(lstm_1.shape)
    print(lstm_2.shape)
    print(lstm_3.shape)
    decoder = Embedded_Decoder(device)
    out_1 = decoder.firstDecoder(lstm_3.squeeze())
    print(out_1.shape)
    in_2 = torch.cat((out_1, lstm_2.squeeze()), 1)
    out_2 = decoder.secondDecoder(in_2)
    print(out_2.shape)
    in_3 = torch.cat((out_2, lstm_1.squeeze()), 1)
    out_3 = decoder.thirdDecoder(in_3)
    print(out_3.shape)
    
    loss = criterion(input_seq[10:, :, :, :].to(device), out_3[10:, :, :, :])
    print(loss)

In [25]:
criterion = nn.MSELoss()
training(criterion)

torch.Size([1, 20, 64, 16, 16])
torch.Size([1, 20, 128, 8, 8])
torch.Size([1, 20, 256, 4, 4])
torch.Size([20, 128, 8, 8])
torch.Size([20, 64, 16, 16])
torch.Size([20, 1, 64, 64])
tensor(0.2782, device='cuda:0', grad_fn=<MseLossBackward0>)


In [ ]:
input_seq = torch.rand(((20, 1, 64, 64)))

loss = criterion(input_)

In [ ]:
decoder.firstDecoder

In [ ]:
decoder